# Charge d'une flotte de véhicules électriques

On s’intéresse dans ce sujet à la charge de plusieurs véhicules électriques connectés à une station de recharge. Chaque véhicule est décrit par une heure et un état de charge d’arrivée et l’utilisateur indique au système d’ordonnancement une heure et un état de charge de récupération. On cherche dans ce contexte à assurer ces demandes, tout en minimisant le coût électrique de recharge et en assurant que la puissance électrique fournie aux différents véhicules ne dépasse pas la puis- sance maximale fournie par le réseau. On s’appuiera sur des données disponibles sur Oasis.

# Cadre d'étude

On discrétise le temps en T points : $(t_i)_{i \in [1,T]}$. Les voitures sont au nombre de N $(voitures_i)_{i \in [1,N]}$.

On se munit également d'un vecteur ligne de coût de l'électricité : $ C := (c_i)_{i \in [1,T]}$ et d'une matrice $Var$ dont l'élément d'indice $(i,j)$ représente la fraction de puissance totale allouée à la voiture $j$ au temps $t_i$

On notera $i_0$ et $i_1$ les temps d'arrivée et de départ de la voiture $i$, ainsi que $Charge_i^0$ et $Charge_i^1$ ses taux de charge à l'arrivée et au départ du centre.

La batterie de la voiture $j$ au temps $i$ sera représentée par l'élément d'indice $(i,j)$  de la matrice $Charge$.

On note enfin $U$ la tension nominale de charge, $P$ la puissance totale du centre.

# Formalisation du problème

## Fonction objectif

La fonction a minimser est la somme des coûts de la puissance réquisitionnée par le centre.

Nous pouvons la représenter par : $f(V) = \sum\limits_{i=1}^N \displaystyle \int_{0}^{T} c(t)*Var_i(t) \, \mathrm{d}t = \sum\limits_{i = 1, j = 1}^{i = N, j = T} c_i*v_{i,j} = sum(c*Var) = \sum\limits_{i=1}^N f_i(Var)$

On on a noté $f_i$ la fonction qui à la matrice $V$ associe le coût de la voiture $i$ sur tout l'intervalle de temps : $f_i(Var) = c*Var^T[i]$

## Contraintes

### Contraintes pré-analyse

Nous avons tout d'abord identifié 6 contraintes dans ce problème, les voici :

 - Puissance max autorisée : $\forall i \in [0,T],  \sum \limits_{j=1}^N v_{i,j} \leq 1$
 - Puissance négative non autorisée : $\forall i \in [0,T], 0 \leq v_{i,j}  $
 - Heure d'arrivée de la voiture $j$ : $t_{j_0} $. Contrainte associée : $v_{{i,j}_{i \leq i_0}} = 0$ 
 - Heure de départ de la voiture $j$ : $t_{j_1} $. Contrainte associée : $v_{{i,j}_{i \ge i_1}} = 0$ 
 - Taux de charge initial. Contrainte associée : $Charge_{i_0,i} = Charge_i^0$ 
 - Taux de charge final. Contrainte associée : $Charge_{i_1,i} = Charge_i^1$ 
 - Modèlisation de la charge de la batterie

### Modélisation de la batterie

Nous avons reçu un échantillon de données représentant le taux de charge d'une batterie et l'intensité de chargement en fonction du temps. A partir de ces données nous avons pu lier la vitesse de charge d'une batterie en fonction de l'intensité de chargement.

![donnees](figures/batterie/affichage_donnees.png)

On repère ici différents palliers d'intensité de charge. Ceci nous a incité à créer le graphique suivant

![donnees](figures/batterie/affichage_donnees_2.png)

On voit ici se dessiner un modèle linéaire, très bruité. Nous avons donc décidé de faire la moyenne de la vitesse de charge pour chaque pallier d'intensité (pour réduire ce bruit), puis d'appliquer la méthode des moindres carrés pour trouver la droite modélisant le mieux cet échantillon de données

![modele](figures/batterie/modele_de_recharge_lstsq_test.png)

On obtient donc la relation suivante :

$Charge_{i+1,j} = Charge_{i,j} + \Delta t * (a * P *\frac{Var_{i,j}}{U} + b)$ avec $a$ le coefficient directeur et $b$ l'ordonnée à l'origine du modèle linéaire. 

On a donc : $Charge_{i_1,i} - Charge_{i_0,i} = \Delta t * P * a * \frac{\sum\limits_{i = i_0}^{i_1} Var_{i,j}}{U} + (i_1 - i_0) * b * \Delta t$ 

Or on a trouvé b de l'ordre de $10^{-3}$ donc négligable ici et on a la nullité de $V_{i,j}$ en dehors des termes de la somme. Ceci est cohérent avec le modèle choisi : si l'intensité est nulle, la voiture au repos voit sa batterie rester constante. Si l'on veut vraiment coller mieux au modèle trouvé, il suffit alors d'augmenter la puissance demandée d'un facteur proportionnel à l'ordonnée à l'origine.

On a donc enfin : $Charge_{i_1,i} - Charge_{i_0,i} = \Delta t * P * a * \frac{\sum\limits_{i = 0}^{T} Var_{i,j}}{U}$

D'où la contrainte simplifiée suivante : $\sum\limits_{i = 0}^{T} Var_{i,j} = U * \frac {\Delta Charge_j}{\Delta t * a * P}$

### Ré-évaluation des contraintes

Les contraintes du problème deviennent alors
  - Puissance maximale autorisée $P$ : $\forall i \in [0,T],  \sum \limits_{j=1}^N Var_{i,j} \leq 1$
  - Heure d'arrivée et de départ des voitures. $\forall j \in [1,N], Var_{{i,j}_{i < j_0}} = 0 = Var_{{i,j}_{i > j_1}}$
  - Charge obligatoire : $\forall j \in [1,N], \sum\limits_{i = 0}^{T} Var_{i,j} = \Delta Charge_j * \frac {U}{\Delta t * a * P} = \Delta Charge_j * K$ avec $K = \frac {U}{\Delta t * a * P}$ constant. 

# Problème final

Finalement, nous en arrivons au problème final, comportant des contraintes individuelles (la charge) et des contraintes couplées (la puissance maximale à ne pas dépasser), tout en prenant en compte le coût des charges:

$$ \min\limits_{\forall j \in [1,N], Var_{{i,j}_{i < j_0}} = 0 = Var_{{i,j}_{i > j_1}} , \forall i \in [0,T],  \sum \limits_{j=1}^N Var_{i,j} \leq 1, \forall j \in [1,N] , \sum\limits_{i = j_0}^{j_1} Var_{i,j} = \Delta Charge_j * K} (f(Var))$$

# Etude du problème et résolution

# Cas d'un unique véhicule

L'étude d'un cas simple tel que le problème à un véhicule est utile afin de comprendre les spécificités de ce problème. Nous allons tout d'abord étudier la fonction objectif, puis l'existence puis l'unicité des solutions de ce problème.

Pour ce cas particulier, on considérera que l'on se place sur l'intervalle de temps correspondant exactement à celui de la charge de la voiture. Cela évite de contraindre la nullité de certaines $v_i$

## Etude de la fonction objectif

On a, pour $Var \in \mathbb R^T, f(Var) = C*Var$

Donc f est linéaire, donc convexe (mais pas fortement).

### Existence et unicité des solutions

Ce problème admettra une solution lorsque les contraintes définissent un domaine non vide. En effet ces contraintes donneraient alors un espace fermé explorable par la fonction objectif, qui est linéaire en dimension finie donc continue.

Le théorème de Weierstrass assure alors l'existence d'une solution au problème de minimisation.

Dans le cas d'un unique véhicule, on a N = 1, ainsi les contraintes se résument à :
 - $\forall i \in [1,T], Var_{i,1} \leq 1$
 - $\sum\limits_{i = 1_0}^{1_1} Var_{i,1} = \Delta Charge_1 * K$
 
 Le problème est donc solvable si et seulement si lorsque l'on maximise la puissance donnée à la voiture on dépasse la contrainte de charge, donc si et seulement si :
 $$ \frac{1_1 - 1_0}{K} \ge \Delta Charge_1 $$

Nous cherchons aussi une condition nécessaire et suffisante afin d'assurer l'unicité d'une solution. On raisonne donc par disjonction de cas. On se place du point de vue d'une solution au problème.

Deux cas de figure se présentent alors:
 - Si aucune charge n'est réalisée sur une heure pleine
 - Si une partie de la charge est réalisée sur une heure creuse
 
Dans le premier cas: 
 - Si la puissance est maximale pendant toute la durée des heures creuses : La solution est unique.
 - Si non, alors on dispose de deux indices de temps en heures creuses pour lesquels la puissance délivrée à la voiture est différente, on peut alors intervertir les puissances de ces deux indices. On a donc le même coût en électricité, mais pas le même profil de charge : La solution n'est pas unique

Dans le second cas, on a nécessairement une puissance maximale sur les heures creuses, car sinon le coût de charge n'est pas optimal. On a également une disjonction de cas semblable au premier cas:
 - Si la puissance est maximale pendant toute la durée des heures pleines : La solution est unique
 - Si non, alors on dispose de deux indices de temps en heures pleines pour lesquels la puissance délivrée à la voiture est différente, on peut alors intervertir les puissances de ces deux indices. On a donc le même coût en électricité, mais pas le même profil de charge : La solution n'est pas unique

Finalement, on a unicité de la solution si et seulement si on a :
$$ \sum\limits_{i \in heures creuses} 1 = \Delta Charge_1 * K $$ 

ou 

$$ 1_1 - 1_0 = \Delta Charge_1 * K$$

Ce qui en pratique n'arrivera pas.

### Résolution du problème

Dans ce cas simple, nous avons occulté les contraintes de nullité sur les $v_i$, comme dit précédemment.
On s'est munis d'un vecteur de coût ayant une valeur basse (1) pour la première moitié du temps, et une valeur haute (2) pour l'autre moitié.

On peut résoudre ce problème par l'algorithme d'Uzawa.

Pour les constantes du problème, nous avons adopté les valeurs suivantes :
 - Puissance totale du centre : $P = 2000 W$
 - Tension de charge unitaire : $U = 400 V$
 - Pente de charge : $a = 0.00184507 A^{-1}$ 

Ceci nous donne un coefficient totale $K = 1083.9697$

Nous avons discrétisé le temps en 10 points, chacun représentant une seconde.

Enfin, nous avons fixé la puissance demandée à 9 en représentation normalisée (c'est à dire dans la même unité que les fractions de puissance allouées à chaque créneau de temps).

### Résultats

L'algorithme a bien fonctionné, voilà le profil de charge que l'on obtient :

Dans l'algorithme d'uzawa nous avons utilisé des paramètres égaux à 1.5

![comparaison](figures/une_voiture/comparaision.png)

Nous avons voulu étudier l'influence des paramètres de l'algorithme d'Uzawa. Cela nous servira plus tard dans le problème à deux voitures.

Voilà les résultats que nous avons obtenu

![influence](figures/une_voiture/influence.png)

Zoom sur la fin de la courbe

![influence](figures/une_voiture/zoom.png)

Le pas semblant optimal semble donc être environ 0.5 nous avons utilisé cette valeur dans la majorité de nos applications suivantes. 

# Cas de deux véhicules

Dans le cas de deux véhicules, on va voir apparaitre des soucis liés à une surexploitation de la puissance disponible. Cela mène à considérer l'algorithme proposé dans le sujet.

Le but de cet algorithme est de distinguer les parties indépendantes d'un problème de minimisation de ses parties interdépendants. Ici les parties indépendantes sont les charges des voitures par rapport à leur objectif individuel, et les parties interdépendantes sont les considérations de puissance maximale allouable

On implémente donc cet algorithme, mais un problème se pose, il est illustré dans le cas suivant

## Problèmes rencontrés

Nous avons voulu tout d'abord tester notre algorithme dans un cas simple : on veut charger deux voitures arrivant et repartant en même temps, sur un temps court et pour la même demande en terme de charge

On a utilisé un prix qui varie entre deux constantes. Une haute : 2 et une faible : 1

Le but était ici d'étudier la réaction de l'algorithme face à une situation très polarisée : les deux voitures ont les mêmes besoins, et cherchent donc toujours les prix les plus bas aux mêmes endroits.

![fail 1](figures/deux_voitures_V1/num_1.png)

On voit sur ce schéma que l'algorithme suit un pallier pendant un certain temps, dont la durée est variable en fonction du paramètres changeant les coefficients de lagranges associés aux contraintes couplées (car il s'opère un équilibrage des valeurs à ce moment là)
Lorsque les coefficients couplés ont trop augmenté la valeur du prix faible par rapport au prix élevé, le système bascule dans l'état opposé : les voitures se chargent majoritairement sur des temps ou le prix est plus élevé

![fail 1](figures/deux_voitures_V1/num_2.png)

Cette "inversion de population" se fait en une seule itération, du fait de la structure de l'algorithme (chaque voiture minimise son prix), et les multiplicateurs de lagrange s'adaptent alors, ce qui donne une éxécution très lente à chaque inversion de population, avec une convergence très lente (le système s'équilibre quand la différence perçue entre les prix hauts et bas est faible, c'est à dire quand les multplicateurs couplés équilibres quasi-parfaitement les prix)

## Analyse de l'erreur et nouvel algorithme

L'erreur a ici été faite en ne considérant pas la puissance allouable à une voiture pour un temps donné comme une variable de décision, nous avons donc modifié l'algorithme proposé pour arriver à un algorithme ayant la même structure (décomposition / coordination), mais la coordination change non pas des multiplicateurs de lagrange mais des variables de décision : les puissances allouables

Un pseudo code de l'algorithme finalement utilisé est donc le suivante ($eps$ est une borne fixée à l'avance, on a en pratique pris $ eps = 10^{-10} $)


While $-eps > Puissance_{consommee} > 1 + eps$:
    
 - décomposition:
    - Résolution de chaque sous problème (changement des puissances allouées, jamais au dessus des puissances allouables)
        
 - coordination:
   - Changement des puissances_allouables

La coordination se déroule de la manière suivante :
 - Evaluation de la surchage sur chaque créneau horaire (si surcharge il y a)
 - Réduction de la surchage sur les créneaux surchargés (on réduit les puissances allouées proportionnellement à la surchage, avec un facteur rho)
 - Si besoin, augmentation des puissances allouables sur les créneaux non surchagés pour garantir que chaque voiture aura assez de puissance disponible pour se charger, on assure ainsi la convergence de l'étape de décomposition

L'implémentation brute est disponible plus loin dans le notebook

On travaille donc avec $4N$ variables de décisions, N étant le nombre de points de discrétisation du temps:
 - $2N$ pour chaque puissance allouée à chaque temps dans les 2 voitures
 - $2N$ pour chaque puissance allouable à chaque temps dans les 2 voitures
 
Et on a en tout $2 (N+1)$ multiplicateurs de Lagrange:
 - $2N$ pour assurer que chaque puissance allouée se situe bien entre 0 et la puissance allouable pour chaque voiture à chaque temps
 - $2$ pour assurer que la somme des puissances allouées au cours de la charge remplie bien la demande de l'utilisateur de chaque voiture
 
Cela implique que notre matrice de décision $Var$ sera donc composée de quatre colonnes de longueur $N$, les deux premières pour les puissances allouées, les deux derniers pour les puissances allouables.

### Exemple précédent

En affichant la même composante que précédemment dans les mêmes conditions, on obtient donc :

![marche 1](figures/deux_voitures_V2/example1.png)

### Exemple plus coriace et implémentation

Mainenant que l'on sait que notre algorithme fonctionne, nous avons voulu le tester dans un cas plus compliqué.
On a donc imposer un prix variable tout au long du temp et contraint les véhicules à  ne se charger que dans une certaines bandes de temps

Pour mettre en place cette interdiction, on a utilisé un fonction $forbiden$, qui nous permet avant même le début de la résolution du problème de mettre en place les fonctions dont nous aurons besoin, en prenant en compte les contraintes de temps

#### Définition des constantes et des variables 

In [13]:
import matplotlib.pyplot as plt
import numpy as np
import time
# Constantes

a = 0.00184507
b = -0.00014853   # Négligé ici (ordonnée à l'origine du modèle de charge de batterie)
P_max = 2000
U = 400           # Tension secteur

x0 = np.array([0 for _ in range(10)], dtype = float)
P_allouable = np.array([1 for _ in range(10)], dtype = float)
lambda0 = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype = float)
Var = np.array([1*x0, 1*x0, 1*P_allouable, 1*P_allouable])

GLOBAL_dt = 1e-1
GLOBAL_Delta_charge = np.array([0.0035, 0.0035])
GLOBAL_K = U/(a*P_max*GLOBAL_dt)

GLOBAL_P_necessaire = GLOBAL_K * GLOBAL_Delta_charge


def cout(t):
    return (1 - t)**2 + 1 

GLOBAL_Cout = np.array([ cout (i*GLOBAL_dt) for i in range(int(1/GLOBAL_dt))])

Viennent ensuite les fonctions utiles au problème, déterminées par la fonction forbiden

#### Définition des fonctions du problème en fonction des contraintes de temps (départ et arrivée)

In [14]:
def forbiden(num, i):
    if num == 0:
        if i < 4:
            return 1
        else:
            return 0
    elif num == 1:
        if i > 7:
            return 1
        else:
            return 0

In [15]:
def transformation_probleme(forbiden, Var):
    global GLOBAL_dt
    global GLOBAL_P_necessaire

    n = len(Var[0])
    # Gradient de f
    Grad_f = []
    for num in range(2):
        Sous_grad = []
        for i in range(n):
            if not forbiden(num, i):
                Sous_grad.append(cout(i*GLOBAL_dt))
            else:
                Sous_grad.append(0)
        Grad_f.append(Sous_grad)

    Grad_f = np.array(Grad_f, dtype = float)

    # Puissance allouable

    for num in range(2):
        for i in range(n):
            if forbiden(num, i):
                Var[2 + num][i] = 0

    # contraintes sur P

    def contraintes_solo(num, X, P):
        GLOBAL_P_necessaire
        contraintes = []
        n = len(X)
        for i in range(n):
            if forbiden(num, i):
                contraintes.append(0)
            else:
                contraintes.append(X[i] * (X[i] - P[i]))

        contraintes.append( - np.sum(X) + GLOBAL_P_necessaire[num])
        return np.array(contraintes)

    def grad_contraintes_solo(num, X, P):
        grad = []
        n = len(X)

        for i in range(n):
            grad.append([0 for _ in range(n)])
            if not forbiden(num, i):
                grad[i][i] = 2*X[i] - P[i]

        grad.append([ -1 for _ in range(n)])
        for i in range(n):
            if forbiden(num, i):
                grad[-1][i] = 0

        return np.array(grad)

    def coordination(Var, rho):
        global GLOBAL_P_necessaire
        n  = len(Var[0])

        ## On évalue tout d'abord la surchage sur chaque créneau horaire
        
        surchage = []

        for i in range(n):
            if Var[0][i] + Var[1][i] >= 1:
                surchage.append( Var[0][i] + Var[1][i] - 1)
            else:
                surchage.append(0)
                
        ## On réduit 
                
        for num in range(2):
            for i in range(n):
                if not forbiden(num, i):
                    Var[num + 2][i] -= rho*surchage[i]

        while ( sum(Var[2]) - GLOBAL_P_necessaire[0] < 0 ) and ( sum(Var[3]) - GLOBAL_P_necessaire[1] < 0 ):
            for num in range(2):
                n_dispos = 0
                indices_dispos = []
                for i in range(n):
                    if not forbiden(num, i) and Var[num + 2][i] != 0 and Var[num + 2][i] != 1:
                        n_dispos += 1
                        indices_dispos.append(i)

            for i in indices_dispos:
                Var[num + 2][i] = min(1 , Var[num+2][i] + (GLOBAL_P_necessaire[num] - sum(Var[num + 2]))/n_dispos)

    return Grad_f, Var, contraintes_solo, grad_contraintes_solo, coordination

grad_f, Var, contraintes_solo, grad_contraintes_solo, coordination = transformation_probleme(forbiden, Var)

On utilise enfin le coeur de notre algorithme : la $decomposition$ (via la méthode d'Uzawa) et la $coordination$

In [16]:
## Comme dans le cas à une voiture, en rajoutant les puissances allouables en paramètre

def Uzawa_1_voiture(Grad_L, c, lambda0, xk, P_max, l, rho, num, max_iter = 1000000, epsilon_grad_L = 1e-3):
    lam = lambda0
    def update_lam(lam, rho, c, xk):
        C = c(num, xk, P_max)
        for i in range(len(lam)):
            lam[i] = max(0, lam[i] + rho*C[i])
    grad_l = Grad_L(lam, xk, P_max, num)
    pk = -1*grad_l
    xk += l*pk
    update_lam(lam, rho, c, xk)
    num_iter = 0

    while num_iter < max_iter and np.linalg.norm(grad_l) > epsilon_grad_L:
        grad_l = Grad_L(lam, xk, P_max, num)
        pk = -1*grad_l
        xk += l*pk
        update_lam(lam, rho, c, xk)
        num_iter += 1

def decomposition(grad_f, c, grad_c, num_max, lam, Var, l, rho_solo):
    def grad_L(lam, X, P, num):
        return grad_f[num] + np.dot(lam,grad_c(num, X, P))
    Uzawa_1_voiture(grad_L, c, lam[0], Var[0], Var[2], l, rho_solo, 0)
    Uzawa_1_voiture(grad_L, c, lam[1], Var[1], Var[3], l, rho_solo, 1)

def decomposition_coordination(grad_f, c_solo, grad_c, num_max, lambda0, Var, l, rho_solo, rho, max_iter = 100_000, eps_diff = 1e-10):
    lam = 1*lambda0
    num_iter = 0
    decomposition(grad_f, c_solo, grad_c, num_max, lam, Var, l, rho_solo)
    Puissances_consommees = Var[0] + Var[1]
    Listes_puissances = [Puissances_consommees]
    # Liste = [[ Var[0][3], Var[1][3] ]]
    coordination(Var, rho)
    while num_iter < max_iter and (Puissances_consommees >= 1 + eps_diff).any() :    
        decomposition(grad_f, c_solo, grad_c, num_max, lam, Var, l, rho_solo)
        temp = 1*lam
        coordination(Var, rho)
        Puissances_consommees = Var[0] + Var[1]
        Listes_puissances.append(Puissances_consommees)
        num_iter += 1
        # Liste.append( [ Var[0][3], Var[1][3] ] )
        print( num_iter)
    return Listes_puissances

On est donc prêts à tester notre exemple plus "coriace", et voilà les résultats

### Résultats

La prochaine figure affiche la somme des puissances allouées au fur et à mesure des itérations de l'algorithme, les numéros représentants les différents points de discrétisation du temps

Les besoins de chaque voiture était d'environ "3", en unité normalisée (c'est à dire la même unité que le pourcentage de puissance allouée sur un point de discrétisation). Du fait de nos temps d'arrivée et de départ, on comprend vite que les voitures seront en compétition sur quelques plages de temps.

Nous avons utilisé des constantes égales à $0.5$ pour :
 - Les pas de la méthode d'Uzawa (celui pour les multiplicateurs et celui pour le pas de descente de gradient)
 - Le pas d'ajustement des puissances allouables en fonction de la surchage

![resultats 2](figures/deux_voitures_V2/beaucoup_ameliore.png)

On observe que les prix les plus faibles sont tous d'abords monopolisés par les deux voitures, puis l'algorithme réduit la puissance allouable sur ces créneaux, la puissance allouée est ensuite reportée sur d'autres créneaux, qui saturent à leur tour, et ainsi de suite jusqu'à ce qu'on atteigne l'équilibre
On converge dans notre exemple en 15 itérations

## Comparaison avec des algorithmes LP

Le problème étant modélisé de manière très particulière et n'ayant pas trouvé d'algorithme pré-programmé pouvant le réduire, nous n'avons pu comparer notre algorithme avec d'autres méthodes...